### Dependencies

In [ ]:
! pip install --upgrade simpletransformers pandas

     |████████████████████████████████| 215kB 4.7MB/s 
     |████████████████████████████████| 10.5MB 14.4MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
     |████████████████████████████████| 890kB 64.7MB/s 
     |████████████████████████████████| 317kB 54.3MB/s 
     |████████████████████████████████| 3.0MB 52.1MB/s 
     |████████████████████████████████| 1.6MB 59.9MB/s 
     |████████████████████████████████| 7.2MB 53.8MB/s 
     |████████████████████████████████| 890kB 47.8MB/s 
     |████████████████████████████████| 1.1MB 50.8MB/s 
     |████████████████████████████████| 102kB 13.7MB/s 
     |████████████████████████████████| 102kB 13.5MB/s 
     |████████████████████████████████| 122kB 60.8MB/s 
     |████████████████████████████████| 163kB 64.0MB/s 
     |████████████████████████████████| 112kB 55.8MB/s 
     |████████████████████████████████| 4.4MB 51.2MB/s 
     |████████████████████████████████| 71kB 12.3MB/s 
     |████████████████████████████████| 122kB 62.5M

### Generating text in SimpleTransformers

In [ ]:
from simpletransformers.language_generation import LanguageGenerationModel
model = LanguageGenerationModel("gpt2", "monsoon-nlp/sanaa")

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
model.generate("مدرستي")

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


['مدرستيان أن نشاك في السبي�']

In [ ]:
model.tokenizer.add_tokens(["[EGYPTIAN]", "[MSA]", "[LEVANTINE]", "[GULF]"])

4

In [ ]:
model.tokenizer.encode("[MSA] مدرستي")

[50258, 149, 227, 148, 107, 148, 109, 148, 111, 148, 103, 149, 232]

## Collect dialect examples
### UBC-NLP
https://github.com/UBC-NLP/aoc_id

In [ ]:
! wget "https://github.com/UBC-NLP/aoc_id/raw/master/data/train/MultiTrain.Shuffled.csv"
! head -n 3 MultiTrain.Shuffled.csv

--2020-09-22 12:55:27--  https://github.com/UBC-NLP/aoc_id/raw/master/data/train/MultiTrain.Shuffled.csv
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/UBC-NLP/aoc_id/master/data/train/MultiTrain.Shuffled.csv [following]
--2020-09-22 12:55:27--  https://raw.githubusercontent.com/UBC-NLP/aoc_id/master/data/train/MultiTrain.Shuffled.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14781520 (14M) [text/plain]
Saving to: ‘MultiTrain.Shuffled.csv’

MultiTrain.Shuffled 100%[===================>]  14.10M  52.9MB/s    in 0.3s    

2020-09-22 12:55:28 (52.9 MB/s) - ‘MultiTrain.Shuffled.csv’ saved [147

In [ ]:
import csv
import random

train = []
test = []
eval = []

dialect = {
    'MSA': '[MSA]',
    'DIAL_EGY': "[EGYPTIAN]",
    'DIAL_LEV': "[LEVANTINE]",
    'DIAL_GLF': "[GULF]"
}

with open("./MultiTrain.Shuffled.csv", "r") as prompts:
  rdr = csv.reader(prompts)
  lines = 0
  for line in rdr:
    lines += 1
    if len(line) == 3 and len(line[0]) > 0:
      if random.random() > 0.4:
        train.append(dialect[line[1]] + line[2] + '<|endoftext|>')
      elif random.random() > 0.2:
        test.append(dialect[line[1]] + line[2] + '<|endoftext|>')
      else:
        eval.append(dialect[line[1]] + line[2] + '<|endoftext|>')

### DART from Qatar University
http://qufaculty.qu.edu.qa/telsayed/datasets/

In [ ]:
! unzip DART.zip

In [ ]:
! head -n 5 DART/cf-data/EGY.txt

﻿score(/3)	tweet_Id	tweet_text
3	848198082374373377	ايه ده ي ابو الاجناش 😂😂😂😂😂😂😂😂😂😂😂
3	848198084110811137	يعني خلاص الشتا خلص و هندخل في الحر و التلزيق و  مش هلبس اللبس اللي مرمي في الدولاب و ملبستوش ده
2	848198084790288386	RT @NouR231323337: قد يأتيك الابتلاء علي هيئة كليه كئيبة معقدة و انت اساساً انسان هايف بيحب الضحك و ملكش في الكلام ده
3	848198085734006784	مكي ده كان ف الزمالك مبيلعبش ب ٢ جنيه و الله


In [ ]:
for dialect in [['EGY', '[EGYPTIAN]'], ['GLF', '[GULF]'], ['LEV', '[LEVANTINE]'], ['MGH', '[MAGHREBI]']]:
  with open("./DART/cf-data/" + dialect[0] + ".txt", "r") as prompts:
    rdr = csv.reader(prompts, delimiter='\t')
    lines = 0
    for line in rdr:
      lines += 1
      if line == 1:
        continue
      if len(line) == 3:
        if random.random() > 0.4:
          train.append(dialect[1] + line[2] + '<|endoftext|>')
        elif random.random() > 0.2:
          test.append(dialect[1] + line[2] + '<|endoftext|>')
        else:
          eval.append(dialect[1] + line[2] + '<|endoftext|>')

### Johns Hopkins LREC
https://github.com/ryancotterell/arabic_dialect_annotation

In [ ]:
! git clone https://github.com/ryancotterell/arabic_dialect_annotation
! gunzip arabic_dialect_annotation/annotated_data.tar.gz
! tar -xvf arabic_dialect_annotation/annotated_data.tar

In [ ]:
! ls annotated_data

egyptian  levantine  README	       twitter-gulf	  twitter-maghrebi
gulf	  maghrebi   README~	       twitter-iraqi
iraqi	  msa	     twitter-egyptian  twitter-levantine


In [ ]:
! head -n 5 annotated_data/gulf

label	text
gulf	
gulf	والله شيء بفقد العقل الله يحميك و يصبر اهلك
gulf	زمان كانو البنات بهالعمر انضج من هلا صح ! بس هلا والله بنت ال15 مش شغل تفتح بيت و تجيب جيل مفيد 
gulf	وحياة الله انو احنا منهبط العزيمت انا برايي انو المنتخب رح يوصل لابعد من الدور الثاني وبدل ها هالسلبية كل واحد يحكيلوا كلمة حلوة ترفع معنويات الشباب


In [ ]:
for dialect in [['egyptian', '[EGYPTIAN]'], ['gulf', '[GULF]'], ['levantine', '[LEVANTINE]'], ['maghrebi', '[MAGHREBI]'], ['msa', '[MSA]']]:
  with open("./annotated_data/" + dialect[0], "r") as prompts:
    rdr = csv.reader(prompts, delimiter='\t')
    lines = 0
    for line in rdr:
      lines += 1
      if line == 1:
        continue
      if len(line) == 2:
        if random.random() > 0.4:
          train.append(dialect[1] + line[1] + '<|endoftext|>')
        elif random.random() > 0.2:
          test.append(dialect[1] + line[1] + '<|endoftext|>')
        else:
          eval.append(dialect[1] + line[1] + '<|endoftext|>')
  
  # with open("./annotated_data/twitter-" + dialect[0], "r") as prompts:
  #     rdr = csv.reader(prompts, delimiter='\t')
  #     lines = 0
  #     for line in rdr:
  #       lines += 1
  #       if line == 1:
  #         continue
  #       if len(line) == 2:
  #         if random.random() > 0.4:
  #           train.append(dialect[1] + line[1] + '<|endoftext|>')
  #         elif random.random() > 0.2:
  #           test.append(dialect[1] + line[1] + '<|endoftext|>')
  #         else:
  #           eval.append(dialect[1] + line[1] + '<|endoftext|>')

In [ ]:
train[-1]

'[MSA]انا مافهمتش ياسى اويحي وياريت لو تفهمنىى لمادا تتشبتون بالكرسى عقود وتأبو ان تتركوه لوعلى جتتكم رغم انكم تعرفون ان الشعب لايوريدكم ورغم انكم اخدتم من المال مايكفيكم انتم وعائلاتكم قرون وليس سنين                                        <|endoftext|>'

In [ ]:
open('./train.txt', 'w').write("\n".join(train))
open('./test.txt', 'w').write("\n".join(test))
open('./eval.txt', 'w').write("\n".join(test))
len(train)

117443

### Finetune model on dialect examples

In [ ]:
from simpletransformers.language_modeling import LanguageModelingModel

train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "train_batch_size": 16, # multiples of 8 are best
    "num_train_epochs": 3,
    "fp16": False,
    "mlm": False,
    "silent": True
}

ft_model = LanguageModelingModel('gpt2', 'monsoon-nlp/sanaa', args=train_args)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


In [ ]:
# dialect tokens
ft_model.tokenizer.add_tokens(["[EGYPTIAN]", "[MSA]", "[LEVANTINE]", "[GULF]", "[MAGHREBI]"])
ft_model.model.resize_token_embeddings(len(ft_model.tokenizer))

Embedding(50262, 768)

In [ ]:
ft_model.train_model("./train.txt", eval_file="./test.txt", silent=True)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [ ]:
ft_model.eval_model("./eval.txt")

{'eval_loss': 1.037823670632743, 'perplexity': tensor(2.8231)}

### Export finetuned model

In [ ]:
ft_model.tokenizer.save_pretrained("./dialects")

('./dialects/vocab.json',
 './dialects/merges.txt',
 './dialects/special_tokens_map.json',
 './dialects/added_tokens.json')

In [ ]:
ft_model.model.save_pretrained("./dialects")

### Generate from finetuned

In [ ]:
from simpletransformers.language_generation import LanguageGenerationModel
model = LanguageGenerationModel("gpt2", "./dialects")

In [ ]:
model.tokenizer.encode('[EGYPTIAN]' + "مدرستي")

[50257, 149, 227, 148, 107, 148, 109, 148, 111, 148, 103, 149, 232]

In [ ]:
model.generate('[EGYPTIAN]' + "مدرستي")

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


['[EGYPTIAN]مدرستي وصبحو خريب']

In [ ]:
model.generate('[GULF]' + "مدينتي هي", { 'max_length': 50 })

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


['[GULF]مدينتي هي اتسبب الحاكم عن المتخصصة ا�']